In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [94]:
pd.set_option("display.max_columns",100)
train_path=r"F:\kaggle\covid-first\train.csv"
test_path=r"F:\kaggle\covid-first\test.csv"

In [95]:
df_train=pd.read_csv(train_path)
df_test=pd.read_csv(test_path)

In [96]:
print(df_train.shape)
print(df_test.shape)  ###############很显然测试机缺少了确诊数以及致死率

(17892, 8)
(12212, 6)


In [97]:
df_train.head() #####训练集数据

,Id,Province/State,Country/Region,Lat,Long,Date,ConfirmedCases,Fatalities
0,1,NaN,Afghanistan,33.0,65.0,2020/1/22,0,0
1,2,NaN,Afghanistan,33.0,65.0,2020/1/23,0,0
2,3,NaN,Afghanistan,33.0,65.0,2020/1/24,0,0
3,4,NaN,Afghanistan,33.0,65.0,2020/1/25,0,0
4,5,NaN,Afghanistan,33.0,65.0,2020/1/26,0,0


In [98]:
df_test.head() ######测试集数据

,ForecastId,Province/State,Country/Region,Lat,Long,Date
0,1,NaN,Afghanistan,33.0,65.0,2020-03-12
1,2,NaN,Afghanistan,33.0,65.0,2020-03-13
2,3,NaN,Afghanistan,33.0,65.0,2020-03-14
3,4,NaN,Afghanistan,33.0,65.0,2020-03-15
4,5,NaN,Afghanistan,33.0,65.0,2020-03-16


--------------------------

1.预处理部分

In [99]:
#######不必要的数据列项规整,国家地区明显可以合并

In [100]:
######将训练集以及测试集一起预处理

In [101]:
df_traintest=pd.concat([df_train,df_test])

In [102]:
df_traintest.shape

(30104, 9)

In [103]:
df_traintest.head()

,Id,Province/State,Country/Region,Lat,Long,Date,ConfirmedCases,Fatalities,ForecastId
0,1.0,NaN,Afghanistan,33.0,65.0,2020/1/22,0.0,0.0,NaN
1,2.0,NaN,Afghanistan,33.0,65.0,2020/1/23,0.0,0.0,NaN
2,3.0,NaN,Afghanistan,33.0,65.0,2020/1/24,0.0,0.0,NaN
3,4.0,NaN,Afghanistan,33.0,65.0,2020/1/25,0.0,0.0,NaN
4,5.0,NaN,Afghanistan,33.0,65.0,2020/1/26,0.0,0.0,NaN


In [104]:
#######很明显这个ID就是个不可能有用的数据

In [105]:
def func(x):
    try:
        x_new=x["Country/Region"]+"/"+x["Province/State"]
    except:
        x_new=x["Country/Region"] 
    return str(x_new)             #####################这一步代码很重要
                                  ####################需要转换类型要不之后无法比较

In [106]:
df_traintest["place_id"]=df_traintest.apply(lambda x:func(x),axis=1)

In [107]:
df_traintest[888:890]     ################成功生成了国家和城市的id

,Id,Province/State,Country/Region,Lat,Long,Date,ConfirmedCases,Fatalities,ForecastId,place_id
888,1309.0,Tasmania,Australia,-41.4545,145.9707,2020/1/28,0.0,0.0,NaN,Australia/Tasmania
889,1310.0,Tasmania,Australia,-41.4545,145.9707,2020/1/29,0.0,0.0,NaN,Australia/Tasmania


In [108]:
tmp=np.sort(df_traintest["place_id"].unique())

In [109]:
print("一共有{}个地区的调查数据".format(len(tmp)))  
print(tmp[:10])

一共有284个地区的调查数据
['Afghanistan' 'Albania' 'Algeria' 'Andorra' 'Antigua and Barbuda'
 'Argentina' 'Armenia' 'Aruba' 'Australia/Australian Capital Territory'
 'Australia/From Diamond Princess']


In [110]:
###############这个日期数据不是很友好，转化一下数据类型

In [111]:
df_traintest["Date"]=pd.to_datetime(df_traintest["Date"])

In [112]:
df_traintest.head()

,Id,Province/State,Country/Region,Lat,Long,Date,ConfirmedCases,Fatalities,ForecastId,place_id
0,1.0,NaN,Afghanistan,33.0,65.0,2020-01-22,0.0,0.0,NaN,Afghanistan
1,2.0,NaN,Afghanistan,33.0,65.0,2020-01-23,0.0,0.0,NaN,Afghanistan
2,3.0,NaN,Afghanistan,33.0,65.0,2020-01-24,0.0,0.0,NaN,Afghanistan
3,4.0,NaN,Afghanistan,33.0,65.0,2020-01-25,0.0,0.0,NaN,Afghanistan
4,5.0,NaN,Afghanistan,33.0,65.0,2020-01-26,0.0,0.0,NaN,Afghanistan


In [115]:
####################插入关于天的数据

In [116]:
df_traintest["day"]=df_traintest["Date"].apply(lambda x: x.dayofyear).astype(np.int16)
df_traintest.head()

,Id,Province/State,Country/Region,Lat,Long,Date,ConfirmedCases,Fatalities,ForecastId,place_id,day
0,1.0,NaN,Afghanistan,33.0,65.0,2020-01-22,0.0,0.0,NaN,Afghanistan,22
1,2.0,NaN,Afghanistan,33.0,65.0,2020-01-23,0.0,0.0,NaN,Afghanistan,23
2,3.0,NaN,Afghanistan,33.0,65.0,2020-01-24,0.0,0.0,NaN,Afghanistan,24
3,4.0,NaN,Afghanistan,33.0,65.0,2020-01-25,0.0,0.0,NaN,Afghanistan,25
4,5.0,NaN,Afghanistan,33.0,65.0,2020-01-26,0.0,0.0,NaN,Afghanistan,26


In [117]:
df_traintest[:-10]

,Id,Province/State,Country/Region,Lat,Long,Date,ConfirmedCases,Fatalities,ForecastId,place_id,day
0,1.0,NaN,Afghanistan,33.0000,65.0000,2020-01-22,0.0,0.0,NaN,Afghanistan,22
1,2.0,NaN,Afghanistan,33.0000,65.0000,2020-01-23,0.0,0.0,NaN,Afghanistan,23
2,3.0,NaN,Afghanistan,33.0000,65.0000,2020-01-24,0.0,0.0,NaN,Afghanistan,24
3,4.0,NaN,Afghanistan,33.0000,65.0000,2020-01-25,0.0,0.0,NaN,Afghanistan,25
4,5.0,NaN,Afghanistan,33.0000,65.0000,2020-01-26,0.0,0.0,NaN,Afghanistan,26
...,...,...,...,...,...,...,...,...,...,...,...
12197,NaN,NaN,Zambia,-15.4167,28.2833,2020-04-09,NaN,NaN,12198.0,Zambia,100
12198,NaN,NaN,Zambia,-15.4167,28.2833,2020-04-10,NaN,NaN,12199.0,Zambia,101
12199,NaN,NaN,Zambia,-15.4167,28.2833,2020-04-11,NaN,NaN,12200.0,Zambia,102
12200,NaN,NaN,Zambia,-15.4167,28.2833,2020-04-12,NaN,NaN,12201.0,Zambia,103


In [118]:
##################可以计算每天的死亡确诊人数了

In [119]:
import copy  #######新的模块,deepcopy是完全独立的复制
places=df_traintest["place_id"].unique()########对每一个地区进行新生追踪

In [120]:
df_traintest2=copy.deepcopy(df_traintest)
df_traintest2["cases/day"]=0
df_traintest2["fatal/day"]=0
for place in places:
    tmp=df_traintest2["ConfirmedCases"][df_traintest2["place_id"]==place].values
    tmp[1:]=tmp[1:]-tmp[:-1]
    df_traintest2["cases/day"][df_traintest2["place_id"]==place]=tmp
    tmp=df_traintest2["Fatalities"][df_traintest2["place_id"]==place].values
    tmp[1:]=tmp[1:]-tmp[:-1]
    df_traintest2["fatal/day"][df_traintest2["place_id"]==place]=tmp

F:\Anaconda\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
F:\Anaconda\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [121]:
df_traintest2[df_traintest2["place_id"]=="China/Hubei"].head()   ##########得到了湖北的感染以及致命人数变化

,Id,Province/State,Country/Region,Lat,Long,Date,ConfirmedCases,Fatalities,ForecastId,place_id,day,cases/day,fatal/day
3780,5581.0,Hubei,China,30.9756,112.2707,2020-01-22,444.0,17.0,NaN,China/Hubei,22,444,17
3781,5582.0,Hubei,China,30.9756,112.2707,2020-01-23,444.0,17.0,NaN,China/Hubei,23,0,0
3782,5583.0,Hubei,China,30.9756,112.2707,2020-01-24,549.0,24.0,NaN,China/Hubei,24,105,7
3783,5584.0,Hubei,China,30.9756,112.2707,2020-01-25,761.0,40.0,NaN,China/Hubei,25,212,16
3784,5585.0,Hubei,China,30.9756,112.2707,2020-01-26,1058.0,52.0,NaN,China/Hubei,26,297,12


In [122]:
##########需要搞清楚什么是窗口函数
#########这里操作一下rolling函数，滚动求平均值

In [123]:
def df_aggregation(df,col,mean_range):
    df_new=copy.deepcopy(df)  
    col_new='{}-({}-{})'.format(col,mean_range[0],mean_range[1])
    df_new[col_new]=0
    tmp=df_new[col].rolling(mean_range[1]-mean_range[0]+1).mean()
    df_new[col_new][mean_range[0]:]=tmp[:-(mean_range[0])]
    df_new[col_new][pd.isna(df_new[col_new])]=0
    return df_new[[col_new]].reset_index(drop=True)  #####################完全把原来的索引丢弃掉，设立新的数字索引

In [124]:
def do_aggregations(df):
    df=(pd.concat([df,df_aggregation(df,'cases/day',[1,1])],axis=1)).reset_index(drop=True)
    df=(pd.concat([df,df_aggregation(df,'cases/day',[1,7])],axis=1)).reset_index(drop=True)
    df=(pd.concat([df,df_aggregation(df,'cases/day',[8,14])],axis=1)).reset_index(drop=True)
    df=(pd.concat([df,df_aggregation(df,'cases/day',[15,21])],axis=1)).reset_index(drop=True)
    df=(pd.concat([df,df_aggregation(df,'fatal/day',[1,1])],axis=1)).reset_index(drop=True)
    df=(pd.concat([df,df_aggregation(df,'fatal/day',[1,7])],axis=1)).reset_index(drop=True)
    df=(pd.concat([df,df_aggregation(df,'fatal/day',[8,14])],axis=1)).reset_index(drop=True)
    df=(pd.concat([df,df_aggregation(df,'fatal/day',[15,21])],axis=1)).reset_index(drop=True)
    for threshold in[1,10,100]: ################设立不同的阈值求和
        days_under_threshold=(df['ConfirmedCases']<threshold).sum()
        tmp=df["day"]-22-days_under_threshold
        tmp[tmp<0]=0                            ###########照顾到22号之前已经爆发的地区，比如中国湖北
        df["days_since_{}cases".format(threshold)]=tmp
    for threshold in [1, 10, 100]:
        days_under_threshold = (df['Fatalities']<threshold).sum()
        tmp = df['day'].values - 22 - days_under_threshold
        tmp[tmp<=0] = 0
        df['days_since_{}fatal'.format(threshold)] = tmp
    if df['place_id'][0]=='China/Hubei':             #################这下面是作为特别爆发去作者补充的武汉数据,作者应该算错日期了
        df['days_since_1cases'] += 35 # 2019/12/8
        df['days_since_10cases'] += 35-13 # 2019/12/8-2020/1/2 assume 2019/12/8+13
        df['days_since_100cases'] += 4 # 2020/1/18
        df['days_since_1fatal'] += 13 # 2020/1/9
    return df

In [125]:
##################分不同地区进行汇总处理

In [126]:
df_traintest3=[]
for place in places:
    df_tmp=df_traintest2[df_traintest2["place_id"]==place].reset_index(drop=True)
    df_tmp=do_aggregations(df_tmp)
    df_traintest3.append(df_tmp)
df_traintest3=pd.concat(df_traintest3).reset_index(drop=True)
df_traintest3[df_traintest3["place_id"]=="China/Hubei"].head()

F:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
F:\Anaconda\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,Id,Province/State,Country/Region,Lat,Long,Date,ConfirmedCases,Fatalities,ForecastId,place_id,day,cases/day,fatal/day,cases/day-(1-1),cases/day-(1-7),cases/day-(8-14),cases/day-(15-21),fatal/day-(1-1),fatal/day-(1-7),fatal/day-(8-14),fatal/day-(15-21),days_since_1cases,days_since_10cases,days_since_100cases,days_since_1fatal,days_since_10fatal,days_since_100fatal
6360,5581.0,Hubei,China,30.9756,112.2707,2020-01-22,444.0,17.0,NaN,China/Hubei,22,444,17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,35,22,4,13,0,0
6361,5582.0,Hubei,China,30.9756,112.2707,2020-01-23,444.0,17.0,NaN,China/Hubei,23,0,0,444.0,0.0,0.0,0.0,17.0,0.0,0.0,0.0,36,23,5,14,1,0
6362,5583.0,Hubei,China,30.9756,112.2707,2020-01-24,549.0,24.0,NaN,China/Hubei,24,105,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37,24,6,15,2,0
6363,5584.0,Hubei,China,30.9756,112.2707,2020-01-25,761.0,40.0,NaN,China/Hubei,25,212,16,105.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,38,25,7,16,3,0
6364,5585.0,Hubei,China,30.9756,112.2707,2020-01-26,1058.0,52.0,NaN,China/Hubei,26,297,12,212.0,0.0,0.0,0.0,16.0,0.0,0.0,0.0,39,26,8,17,4,0


In [127]:
##################作者开始引入新的数据,这是数据分析一个亮点，引入了不同国家吸烟率，吸烟肺功能下降，可能影响肺炎传播
##################艾滋病率,政府年卫生投入资金都是有关可以考虑的数据
##################吸烟数据:https://ourworldindata.org/smoking
##################但是数据表似乎更新了，我从作者的kaggle数据库中找到了这张表

In [128]:
smoke_path=r"F:\kaggle\covid-first\share-of-adults-who-smoke.csv"

In [129]:
df_smoking=pd.read_csv(smoke_path)

In [130]:
df_smoking.head()

,Entity,Code,Year,"Smoking prevalence, total (ages 15+) (% of adults)"
0,Albania,ALB,2000,34.8
1,Albania,ALB,2005,32.7
2,Albania,ALB,2010,31.2
3,Albania,ALB,2011,30.7
4,Albania,ALB,2012,30.2


In [131]:
df_smoking.head()
print(np.sort(df_smoking['Entity'].unique())[:15])  #############排前面的吸烟地区

['Albania' 'Algeria' 'Andorra' 'Arab World' 'Argentina' 'Armenia'
 'Australia' 'Austria' 'Azerbaijan' 'Bahamas' 'Bahrain' 'Bangladesh'
 'Barbados' 'Belarus' 'Belgium']


In [132]:
##########抽取新的数据，只要最新的

In [133]:
##########倒序排列

In [134]:
df_smoking_recent=df_smoking.sort_values("Year",ascending=False).reset_index(drop=True) 
df_smoking_recent=df_smoking_recent[df_smoking_recent["Entity"].duplicated()==False]
###########################改了两列的名字,没什么大变动
df_smoking_recent['Country/Region'] = df_smoking_recent['Entity']
df_smoking_recent['SmokingRate'] = df_smoking_recent['Smoking prevalence, total (ages 15+) (% of adults)']
df_smoking_recent.head()
####################最新竟然只有16年,这个机构在摸鱼

,Entity,Code,Year,"Smoking prevalence, total (ages 15+) (% of adults)",Country/Region,SmokingRate
0,Zimbabwe,ZWE,2016,15.8,Zimbabwe,15.8
1,Namibia,NAM,2016,21.4,Namibia,21.4
2,Suriname,SUR,2016,25.0,Suriname,25.0
3,Morocco,MAR,2016,23.4,Morocco,23.4
4,India,IND,2016,11.5,India,11.5


In [135]:
##############合并新的数据

In [136]:
df_traintest4 = pd.merge(df_traintest3,df_smoking_recent[["Country/Region","SmokingRate"]],on="Country/Region",how="left")
print(df_traintest4.shape)

(30104, 28)


In [137]:
df_traintest4.head() ########前面几项都是NaN,代表吸烟率表格中没有数据相匹配

,Id,Province/State,Country/Region,Lat,Long,Date,ConfirmedCases,Fatalities,ForecastId,place_id,day,cases/day,fatal/day,cases/day-(1-1),cases/day-(1-7),cases/day-(8-14),cases/day-(15-21),fatal/day-(1-1),fatal/day-(1-7),fatal/day-(8-14),fatal/day-(15-21),days_since_1cases,days_since_10cases,days_since_100cases,days_since_1fatal,days_since_10fatal,days_since_100fatal,SmokingRate
0,1.0,NaN,Afghanistan,33.0,65.0,2020-01-22,0.0,0.0,NaN,Afghanistan,22,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,NaN
1,2.0,NaN,Afghanistan,33.0,65.0,2020-01-23,0.0,0.0,NaN,Afghanistan,23,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,NaN
2,3.0,NaN,Afghanistan,33.0,65.0,2020-01-24,0.0,0.0,NaN,Afghanistan,24,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,NaN
3,4.0,NaN,Afghanistan,33.0,65.0,2020-01-25,0.0,0.0,NaN,Afghanistan,25,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,NaN
4,5.0,NaN,Afghanistan,33.0,65.0,2020-01-26,0.0,0.0,NaN,Afghanistan,26,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,NaN


In [138]:
#######################用世界平均吸烟率去代替空缺的吸烟率

In [139]:
SmokingRate=df_smoking_recent["SmokingRate"][df_smoking_recent["Entity"]=="World"].values[0]
df_traintest4["SmokingRate"][pd.isna(df_traintest4["SmokingRate"])]=SmokingRate
df_traintest4.head()

F:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Id,Province/State,Country/Region,Lat,Long,Date,ConfirmedCases,Fatalities,ForecastId,place_id,day,cases/day,fatal/day,cases/day-(1-1),cases/day-(1-7),cases/day-(8-14),cases/day-(15-21),fatal/day-(1-1),fatal/day-(1-7),fatal/day-(8-14),fatal/day-(15-21),days_since_1cases,days_since_10cases,days_since_100cases,days_since_1fatal,days_since_10fatal,days_since_100fatal,SmokingRate
0,1.0,NaN,Afghanistan,33.0,65.0,2020-01-22,0.0,0.0,NaN,Afghanistan,22,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,20.475517
1,2.0,NaN,Afghanistan,33.0,65.0,2020-01-23,0.0,0.0,NaN,Afghanistan,23,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,20.475517
2,3.0,NaN,Afghanistan,33.0,65.0,2020-01-24,0.0,0.0,NaN,Afghanistan,24,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,20.475517
3,4.0,NaN,Afghanistan,33.0,65.0,2020-01-25,0.0,0.0,NaN,Afghanistan,25,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,20.475517
4,5.0,NaN,Afghanistan,33.0,65.0,2020-01-26,0.0,0.0,NaN,Afghanistan,26,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,20.475517


In [140]:
#################作者又去寻找了新的数据
#################https://www.imf.org/external/pubs/ft/weo/2017/01/weodata/index.aspx

In [141]:
smoke_path=r"F:\kaggle\covid-first\WEO.csv"

In [142]:
df_weo=pd.read_csv(smoke_path)

In [143]:
df_weo.head()        #############展示新的数据，作者找了部分国家的经济数据作为分析出发点

,Country,Subject Descriptor,Units,Scale,Country/Series-specific Notes,2015,2016,2017,2018,2019,2020,2021,2022,Estimates Start After
0,Afghanistan,"Gross domestic product, constant prices",National currency,Billions,Source: National Statistics Office Latest actu...,467.035,476.376,490.667,507.84,528.154,551.921,579.517,611.39,2014.0
1,Afghanistan,"Gross domestic product, constant prices",Percent change,NaN,"See notes for: Gross domestic product, consta...",0.758,2,3,3.5,4,4.5,5,5.5,2014.0
2,Afghanistan,"Gross domestic product, current prices",National currency,Billions,Source: National Statistics Office Latest actu...,"1,203.67","1,281.66","1,399.43","1,535.31","1,692.57","1,874.47","2,085.38","2,330.54",2014.0
3,Afghanistan,"Gross domestic product, current prices",U.S. dollars,Billions,"See notes for: Gross domestic product, curren...",19.687,18.886,20.57,21.706,23.233,24.93,26.857,29.113,2014.0
4,Afghanistan,"Gross domestic product, deflator",Index,NaN,"See notes for: Gross domestic product, consta...",257.726,269.044,285.209,302.321,320.469,339.627,359.848,381.188,2014.0


In [144]:
print(df_weo['Subject Descriptor'].unique())  #################累了，累了，去跑步了

['Gross domestic product, constant prices'
 'Gross domestic product, current prices'
 'Gross domestic product, deflator'
 'Gross domestic product per capita, constant prices'
 'Gross domestic product per capita, current prices'
 'Output gap in percent of potential GDP'
 'Gross domestic product based on purchasing-power-parity (PPP) valuation of country GDP'
 'Gross domestic product based on purchasing-power-parity (PPP) per capita GDP'
 'Gross domestic product based on purchasing-power-parity (PPP) share of world total'
 'Implied PPP conversion rate' 'Total investment' 'Gross national savings'
 'Inflation, average consumer prices'
 'Inflation, end of period consumer prices'
 'Six-month London interbank offered rate (LIBOR)'
 'Volume of imports of goods and services' 'Volume of Imports of goods'
 'Volume of exports of goods and services' 'Volume of exports of goods'
 'Unemployment rate' 'Employment' 'Population'
 'General government revenue' 'General government total expenditure'
 'Gene

In [205]:
subs=df_weo["Subject Descriptor"].unique()[:-1]  ###########去掉最后一个空缺值
df_weo_agg=df_weo[["Country"]][df_weo["Country"].duplicated()==False].reset_index(drop=True)
for sub in subs[:]:
    df_tmp=df_weo[["Country","2019"]][df_weo["Subject Descriptor"]==sub].reset_index(drop=True)
    df_tmp=df_tmp[df_tmp["Country"].duplicated()==False].reset_index(drop=True)
    df_tmp.columns=["Country",sub]          ##############把表头的2019改了
    df_weo_agg=df_weo_agg.merge(df_tmp,on="Country",how="left")
df_weo_agg.columns=["".join(c if c.isalnum() else "_" for c in str(x))for x in df_weo_agg.columns]
df_weo_agg.columns
df_weo_agg['Country/Region'] = df_weo_agg['Country']
df_weo_agg.head()        #####################各个经济指标的数据

,Country,Gross_domestic_product__constant_prices,Gross_domestic_product__current_prices,Gross_domestic_product__deflator,Gross_domestic_product_per_capita__constant_prices,Gross_domestic_product_per_capita__current_prices,Output_gap_in_percent_of_potential_GDP,Gross_domestic_product_based_on_purchasing_power_parity__PPP__valuation_of_country_GDP,Gross_domestic_product_based_on_purchasing_power_parity__PPP__per_capita_GDP,Gross_domestic_product_based_on_purchasing_power_parity__PPP__share_of_world_total,Implied_PPP_conversion_rate,Total_investment,Gross_national_savings,Inflation__average_consumer_prices,Inflation__end_of_period_consumer_prices,Six_month_London_interbank_offered_rate__LIBOR_,Volume_of_imports_of_goods_and_services,Volume_of_Imports_of_goods,Volume_of_exports_of_goods_and_services,Volume_of_exports_of_goods,Unemployment_rate,Employment,Population,General_government_revenue,General_government_total_expenditure,General_government_net_lending_borrowing,General_government_structural_balance,General_government_primary_net_lending_borrowing,General_government_net_debt,General_government_gross_debt,Gross_domestic_product_corresponding_to_fiscal_year__current_prices,Current_account_balance,Country/Region
0,Afghanistan,528.154,"1,692.57",320.469,"14,300.19","45,827.64",NaN,75.87,"2,054.23",0.053,22.309,18.346,20.174,124.809,131.29,NaN,3.559,2.773,13.252,17.026,NaN,NaN,36.933,469.79,467.62,2.17,NaN,2.532,NaN,101.669,"1,692.57",0.425,Afghanistan
1,Albania,858.595,"1,841.87",214.522,"299,027.69","641,479.81",NaN,40.999,"14,278.81",0.029,44.925,26.646,14.854,130.977,132.641,NaN,2.479,0.836,4.043,4.242,15.35,NaN,2.871,492.277,537.234,-44.957,NaN,-6.302,"1,113.30","1,113.30","1,841.87",-1.632,Albania
2,Algeria,"7,583.96","21,125.43",278.554,"176,011.62","490,287.91",NaN,675.691,"15,681.72",0.475,31.265,38.873,29.337,208.827,211.975,NaN,-2.309,-2.309,1.649,1.79,14.521,NaN,43.088,"6,365.52","6,550.00",-184.478,NaN,-287.307,"3,710.77","3,710.77","21,125.43",-17.345,Algeria
3,Angola,"1,760.49","27,359.28","1,554.07","58,886.08","915,131.64",NaN,208.583,"6,976.84",0.147,131.167,7.284,3.555,337.704,355.519,NaN,1.543,1.556,0.424,0.186,NaN,NaN,29.897,"5,304.57","6,265.99",-961.422,NaN,-54.413,NaN,"17,195.39","27,359.28",-5.153,Angola
4,Antigua and Barbuda,2.919,4.26,145.942,"31,317.40","45,705.31",NaN,2.563,"27,500.00",0.002,1.662,NaN,15.336,143.235,144.893,NaN,0.777,1.7,2.386,1.665,NaN,NaN,0.093,0.921,0.905,0.017,NaN,0.126,NaN,3.595,4.26,-0.152,Antigua and Barbuda


In [146]:
####################合并新的数据

In [206]:
df_traintest5 = pd.merge(df_traintest4, df_weo_agg, on='Country/Region', how='left')
print(df_traintest5.shape)
df_traintest5.head()      #####################有空缺值很正常，很粗的数据

(30104, 60)


,Id,Province/State,Country/Region,Lat,Long,Date,ConfirmedCases,Fatalities,ForecastId,place_id,day,cases/day,fatal/day,cases/day-(1-1),cases/day-(1-7),cases/day-(8-14),cases/day-(15-21),fatal/day-(1-1),fatal/day-(1-7),fatal/day-(8-14),fatal/day-(15-21),days_since_1cases,days_since_10cases,days_since_100cases,days_since_1fatal,days_since_10fatal,days_since_100fatal,SmokingRate,Country,Gross_domestic_product__constant_prices,Gross_domestic_product__current_prices,Gross_domestic_product__deflator,Gross_domestic_product_per_capita__constant_prices,Gross_domestic_product_per_capita__current_prices,Output_gap_in_percent_of_potential_GDP,Gross_domestic_product_based_on_purchasing_power_parity__PPP__valuation_of_country_GDP,Gross_domestic_product_based_on_purchasing_power_parity__PPP__per_capita_GDP,Gross_domestic_product_based_on_purchasing_power_parity__PPP__share_of_world_total,Implied_PPP_conversion_rate,Total_investment,Gross_national_savings,Inflation__average_consumer_prices,Inflation__end_of_period_consumer_prices,Six_month_London_interbank_offered_rate__LIBOR_,Volume_of_imports_of_goods_and_services,Volume_of_Imports_of_goods,Volume_of_exports_of_goods_and_services,Volume_of_exports_of_goods,Unemployment_rate,Employment,Population,General_government_revenue,General_government_total_expenditure,General_government_net_lending_borrowing,General_government_structural_balance,General_government_primary_net_lending_borrowing,General_government_net_debt,General_government_gross_debt,Gross_domestic_product_corresponding_to_fiscal_year__current_prices,Current_account_balance
0,1.0,NaN,Afghanistan,33.0,65.0,2020-01-22,0.0,0.0,NaN,Afghanistan,22,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,20.475517,Afghanistan,528.154,"1,692.57",320.469,"14,300.19","45,827.64",NaN,75.87,"2,054.23",0.053,22.309,18.346,20.174,124.809,131.29,NaN,3.559,2.773,13.252,17.026,NaN,NaN,36.933,469.79,467.62,2.17,NaN,2.532,NaN,101.669,"1,692.57",0.425
1,2.0,NaN,Afghanistan,33.0,65.0,2020-01-23,0.0,0.0,NaN,Afghanistan,23,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,20.475517,Afghanistan,528.154,"1,692.57",320.469,"14,300.19","45,827.64",NaN,75.87,"2,054.23",0.053,22.309,18.346,20.174,124.809,131.29,NaN,3.559,2.773,13.252,17.026,NaN,NaN,36.933,469.79,467.62,2.17,NaN,2.532,NaN,101.669,"1,692.57",0.425
2,3.0,NaN,Afghanistan,33.0,65.0,2020-01-24,0.0,0.0,NaN,Afghanistan,24,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,20.475517,Afghanistan,528.154,"1,692.57",320.469,"14,300.19","45,827.64",NaN,75.87,"2,054.23",0.053,22.309,18.346,20.174,124.809,131.29,NaN,3.559,2.773,13.252,17.026,NaN,NaN,36.933,469.79,467.62,2.17,NaN,2.532,NaN,101.669,"1,692.57",0.425
3,4.0,NaN,Afghanistan,33.0,65.0,2020-01-25,0.0,0.0,NaN,Afghanistan,25,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,20.475517,Afghanistan,528.154,"1,692.57",320.469,"14,300.19","45,827.64",NaN,75.87,"2,054.23",0.053,22.309,18.346,20.174,124.809,131.29,NaN,3.559,2.773,13.252,17.026,NaN,NaN,36.933,469.79,467.62,2.17,NaN,2.532,NaN,101.669,"1,692.57",0.425
4,5.0,NaN,Afghanistan,33.0,65.0,2020-01-26,0.0,0.0,NaN,Afghanistan,26,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,20.475517,Afghanistan,528.154,"1,692.57",320.469,"14,300.19","45,827.64",NaN,75.87,"2,054.23",0.053,22.309,18.346,20.174,124.809,131.29,NaN,3.559,2.773,13.252,17.026,NaN,NaN,36.933,469.79,467.62,2.17,NaN,2.532,NaN,101.669,"1,692.57",0.425


In [148]:
#####################引入新的变量,人均寿命

In [207]:
df_life = pd.read_csv("F:\kaggle\covid-first\Life expectancy at birth.csv")

In [208]:
df_life.head()  #####################筛选我们需要的数据

,HDI Rank (2018),Country,1990,Unnamed: 3,1991,Unnamed: 5,1992,Unnamed: 7,1993,Unnamed: 9,1994,Unnamed: 11,1995,Unnamed: 13,1996,Unnamed: 15,1997,Unnamed: 17,1998,Unnamed: 19,1999,Unnamed: 21,2000,Unnamed: 23,2001,Unnamed: 25,2002,Unnamed: 27,2003,Unnamed: 29,2004,Unnamed: 31,2005,Unnamed: 33,2006,Unnamed: 35,2007,Unnamed: 37,2008,Unnamed: 39,2009,Unnamed: 41,2010,Unnamed: 43,2011,Unnamed: 45,2012,Unnamed: 47,2013,Unnamed: 49,2014,Unnamed: 51,2015,Unnamed: 53,2016,Unnamed: 55,2017,Unnamed: 57,2018,Unnamed: 59
0,170,Afghanistan,50.3,NaN,51,NaN,51.6,NaN,52.3,NaN,52.8,NaN,53.4,NaN,53.9,NaN,54.4,NaN,54.9,NaN,55.4,NaN,55.8,NaN,56.3,NaN,56.8,NaN,57.3,NaN,57.8,NaN,58.3,NaN,58.8,NaN,59.4,NaN,59.9,NaN,60.5,NaN,61,NaN,61.6,NaN,62.1,NaN,62.5,NaN,63,NaN,63.4,NaN,63.8,NaN,64.1,NaN,64.5,NaN
1,69,Albania,71.8,NaN,71.8,NaN,71.8,NaN,71.9,NaN,72,NaN,72.2,NaN,72.5,NaN,72.8,NaN,73.2,NaN,73.6,NaN,74,NaN,74.3,NaN,74.6,NaN,74.8,NaN,75,NaN,75.2,NaN,75.4,NaN,75.6,NaN,75.9,NaN,76.2,NaN,76.6,NaN,76.9,NaN,77.3,NaN,77.6,NaN,77.8,NaN,78,NaN,78.2,NaN,78.3,NaN,78.5,NaN
2,82,Algeria,66.9,NaN,67.3,NaN,67.6,NaN,67.9,NaN,68.2,NaN,68.5,NaN,68.9,NaN,69.3,NaN,69.7,NaN,70.2,NaN,70.6,NaN,71.1,NaN,71.6,NaN,72.1,NaN,72.6,NaN,73.1,NaN,73.5,NaN,73.9,NaN,74.3,NaN,74.6,NaN,74.9,NaN,75.2,NaN,75.4,NaN,75.7,NaN,75.9,NaN,76.1,NaN,76.3,NaN,76.5,NaN,76.7,NaN
3,36,Andorra,76.5,NaN,76.7,NaN,76.9,NaN,77,NaN,77.2,NaN,77.4,NaN,77.6,NaN,77.9,NaN,78.2,NaN,78.5,NaN,78.9,NaN,79.2,NaN,79.5,NaN,79.8,NaN,80,NaN,80.2,NaN,80.3,NaN,80.5,NaN,80.6,NaN,80.7,NaN,80.8,NaN,80.9,NaN,81.1,NaN,81.2,NaN,81.3,NaN,81.4,NaN,81.5,NaN,81.7,NaN,81.8,a
4,149,Angola,45.3,NaN,45.3,NaN,45.2,NaN,45.2,NaN,45.2,NaN,45.2,NaN,45.4,NaN,45.5,NaN,45.8,NaN,46.1,NaN,46.5,NaN,47.1,NaN,47.7,NaN,48.4,NaN,49.3,NaN,50.2,NaN,51.1,NaN,52.2,NaN,53.2,NaN,54.3,NaN,55.4,NaN,56.3,NaN,57.2,NaN,58.1,NaN,58.8,NaN,59.4,NaN,59.9,NaN,60.4,NaN,60.8,NaN


In [209]:
tmp=df_life.iloc[:,1].values.tolist()
df_life=df_life[["Country","2018"]]
def func(x):
    try:
        x_new=float(x.replace(",",""))
    except:
        print(x)
        x_new=np.nan
    return x_new
df_life["2018"]=df_life["2018"].apply(lambda x:func(x))
df_life.head()

..
..
nan


,Country,2018
0,Afghanistan,64.5
1,Albania,78.5
2,Algeria,76.7
3,Andorra,81.8
4,Angola,60.8


In [210]:
df_life = df_life[['Country', '2018']]
df_life.columns = ['Country/Region', 'LifeExpectancy']  #############表头转换

In [211]:
df_traintest6 = pd.merge(df_traintest5, df_life, on='Country/Region', how='left')
print(len(df_traintest6))
df_traintest6.head()

30104


,Id,Province/State,Country/Region,Lat,Long,Date,ConfirmedCases,Fatalities,ForecastId,place_id,day,cases/day,fatal/day,cases/day-(1-1),cases/day-(1-7),cases/day-(8-14),cases/day-(15-21),fatal/day-(1-1),fatal/day-(1-7),fatal/day-(8-14),fatal/day-(15-21),days_since_1cases,days_since_10cases,days_since_100cases,days_since_1fatal,days_since_10fatal,days_since_100fatal,SmokingRate,Country,Gross_domestic_product__constant_prices,Gross_domestic_product__current_prices,Gross_domestic_product__deflator,Gross_domestic_product_per_capita__constant_prices,Gross_domestic_product_per_capita__current_prices,Output_gap_in_percent_of_potential_GDP,Gross_domestic_product_based_on_purchasing_power_parity__PPP__valuation_of_country_GDP,Gross_domestic_product_based_on_purchasing_power_parity__PPP__per_capita_GDP,Gross_domestic_product_based_on_purchasing_power_parity__PPP__share_of_world_total,Implied_PPP_conversion_rate,Total_investment,Gross_national_savings,Inflation__average_consumer_prices,Inflation__end_of_period_consumer_prices,Six_month_London_interbank_offered_rate__LIBOR_,Volume_of_imports_of_goods_and_services,Volume_of_Imports_of_goods,Volume_of_exports_of_goods_and_services,Volume_of_exports_of_goods,Unemployment_rate,Employment,Population,General_government_revenue,General_government_total_expenditure,General_government_net_lending_borrowing,General_government_structural_balance,General_government_primary_net_lending_borrowing,General_government_net_debt,General_government_gross_debt,Gross_domestic_product_corresponding_to_fiscal_year__current_prices,Current_account_balance,LifeExpectancy
0,1.0,NaN,Afghanistan,33.0,65.0,2020-01-22,0.0,0.0,NaN,Afghanistan,22,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,20.475517,Afghanistan,528.154,"1,692.57",320.469,"14,300.19","45,827.64",NaN,75.87,"2,054.23",0.053,22.309,18.346,20.174,124.809,131.29,NaN,3.559,2.773,13.252,17.026,NaN,NaN,36.933,469.79,467.62,2.17,NaN,2.532,NaN,101.669,"1,692.57",0.425,64.5
1,2.0,NaN,Afghanistan,33.0,65.0,2020-01-23,0.0,0.0,NaN,Afghanistan,23,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,20.475517,Afghanistan,528.154,"1,692.57",320.469,"14,300.19","45,827.64",NaN,75.87,"2,054.23",0.053,22.309,18.346,20.174,124.809,131.29,NaN,3.559,2.773,13.252,17.026,NaN,NaN,36.933,469.79,467.62,2.17,NaN,2.532,NaN,101.669,"1,692.57",0.425,64.5
2,3.0,NaN,Afghanistan,33.0,65.0,2020-01-24,0.0,0.0,NaN,Afghanistan,24,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,20.475517,Afghanistan,528.154,"1,692.57",320.469,"14,300.19","45,827.64",NaN,75.87,"2,054.23",0.053,22.309,18.346,20.174,124.809,131.29,NaN,3.559,2.773,13.252,17.026,NaN,NaN,36.933,469.79,467.62,2.17,NaN,2.532,NaN,101.669,"1,692.57",0.425,64.5
3,4.0,NaN,Afghanistan,33.0,65.0,2020-01-25,0.0,0.0,NaN,Afghanistan,25,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,20.475517,Afghanistan,528.154,"1,692.57",320.469,"14,300.19","45,827.64",NaN,75.87,"2,054.23",0.053,22.309,18.346,20.174,124.809,131.29,NaN,3.559,2.773,13.252,17.026,NaN,NaN,36.933,469.79,467.62,2.17,NaN,2.532,NaN,101.669,"1,692.57",0.425,64.5
4,5.0,NaN,Afghanistan,33.0,65.0,2020-01-26,0.0,0.0,NaN,Afghanistan,26,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,20.475517,Afghanistan,528.154,"1,692.57",320.469,"14,300.19","45,827.64",NaN,75.87,"2,054.23",0.053,22.309,18.346,20.174,124.809,131.29,NaN,3.559,2.773,13.252,17.026,NaN,NaN,36.933,469.79,467.62,2.17,NaN,2.532,NaN,101.669,"1,692.57",0.425,64.5


In [212]:
#########################引入特殊的数据
########################总结一下数据大概就是：
########################城乡人口,群众聚会限制，各年龄段的性别,测温仪基础设施等等

In [264]:
df_country = pd.read_csv("F:\kaggle\covid-first\covid19countryinfo.csv")[["country","pop","tests","testpop","density","medianage","urbanpop","quarantine","schools","hospibed","smokers","sex0","sex14","sex25","sex54","sex64","sex65plus","sexratio","lung","femalelung","malelung"]]
df_country.head() 

,country,pop,tests,testpop,density,medianage,urbanpop,quarantine,schools,hospibed,smokers,sex0,sex14,sex25,sex54,sex64,sex65plus,sexratio,lung,femalelung,malelung
0,Afghanistan,"38,928,346",NaN,NaN,60.0,18.0,25.0,NaN,NaN,0.5,NaN,1.05,1.03,1.03,1.03,0.97,0.85,1.03,37.62,36.31,39.33
1,Albania,"2,877,797",NaN,NaN,105.0,36.0,63.0,NaN,NaN,2.9,29.4,1.08,1.11,1.09,0.93,0.95,0.87,0.98,11.67,7.02,17.04
2,Algeria,"43,851,044",NaN,NaN,18.0,29.0,73.0,NaN,NaN,1.9,NaN,1.05,1.05,1.05,1.03,1.01,0.89,1.03,8.77,5.03,12.81
3,Andorra,"77,265",NaN,NaN,164.0,45.0,88.0,NaN,NaN,2.5,32.5,1.07,1.05,1.08,1.05,1.15,1.02,1.06,NaN,NaN,NaN
4,Antigua and Barbuda,"97,929",NaN,NaN,223.0,34.0,26.0,NaN,NaN,3.8,NaN,1.05,1.03,0.99,0.84,0.82,0.76,0.90,11.76,7.67,18.78


In [265]:
df_country["femalelung"][df_country["country"]=="China"]

33     56.35
110      NaN
192      NaN
193      NaN
Name: femalelung, dtype: float64

In [266]:
df_country["Country/Region"]=df_country["country"]
df_country=df_country[df_country["country"].duplicated()==False]

In [267]:
print(df_country[df_country['country'].duplicated()].shape)   ############作者"restrictions"这个属性怎么来的？十分奇怪                                                              ############作者下面合并的键类型错误了

(0, 22)


In [268]:
df_country["femalelung"][df_country["Country/Region"]=="China"]

33    56.35
Name: femalelung, dtype: float64

In [269]:
df_traintest7=pd.merge(df_traintest6,df_country.drop(["country","testpop","tests"],axis=1),on=["Country/Region"],how="left")
print(df_traintest7.shape)
df_traintest7.head()
df_traintest7[df_traintest7["Country/Region"]=="China"]

(30104, 79)


,Id,Province/State,Country/Region,Lat,Long,Date,ConfirmedCases,Fatalities,ForecastId,place_id,day,cases/day,fatal/day,cases/day-(1-1),cases/day-(1-7),cases/day-(8-14),cases/day-(15-21),fatal/day-(1-1),fatal/day-(1-7),fatal/day-(8-14),fatal/day-(15-21),days_since_1cases,days_since_10cases,days_since_100cases,days_since_1fatal,days_since_10fatal,days_since_100fatal,SmokingRate,Country,Gross_domestic_product__constant_prices,Gross_domestic_product__current_prices,Gross_domestic_product__deflator,Gross_domestic_product_per_capita__constant_prices,Gross_domestic_product_per_capita__current_prices,Output_gap_in_percent_of_potential_GDP,Gross_domestic_product_based_on_purchasing_power_parity__PPP__valuation_of_country_GDP,Gross_domestic_product_based_on_purchasing_power_parity__PPP__per_capita_GDP,Gross_domestic_product_based_on_purchasing_power_parity__PPP__share_of_world_total,Implied_PPP_conversion_rate,Total_investment,Gross_national_savings,Inflation__average_consumer_prices,Inflation__end_of_period_consumer_prices,Six_month_London_interbank_offered_rate__LIBOR_,Volume_of_imports_of_goods_and_services,Volume_of_Imports_of_goods,Volume_of_exports_of_goods_and_services,Volume_of_exports_of_goods,Unemployment_rate,Employment,Population,General_government_revenue,General_government_total_expenditure,General_government_net_lending_borrowing,General_government_structural_balance,General_government_primary_net_lending_borrowing,General_government_net_debt,General_government_gross_debt,Gross_domestic_product_corresponding_to_fiscal_year__current_prices,Current_account_balance,LifeExpectancy,pop,density,medianage,urbanpop,quarantine,schools,hospibed,smokers,sex0,sex14,sex25,sex54,sex64,sex65plus,sexratio,lung,femalelung,malelung
4982,4372.0,Anhui,China,31.8257,117.2264,2020-01-22,1.0,0.0,NaN,China/Anhui,22,1,0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0,0,0,0,0,0,25.6,China,"89,473.25","95,199.40",106.4,"63,579.45","67,648.44",NaN,"27,272.31","19,379.63",19.191,3.491,42.674,43.9,125.931,123.888,NaN,3.877,3.6,2.689,3,4.02,NaN,"1,407.27","26,023.24","29,245.77","-3,222.53","-3,233.26","-2,116.07",NaN,"51,791.90","95,199.40",170.037,76.7,"1,439,323,776",153.0,38.0,61.0,NaN,1/26/2020,4.2,24.7,1.15,1.17,1.14,1.04,1.02,0.92,1.06,63.1,56.35,70.52
4983,4373.0,Anhui,China,31.8257,117.2264,2020-01-23,9.0,0.0,NaN,China/Anhui,23,8,0,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1,0,0,0,0,0,25.6,China,"89,473.25","95,199.40",106.4,"63,579.45","67,648.44",NaN,"27,272.31","19,379.63",19.191,3.491,42.674,43.9,125.931,123.888,NaN,3.877,3.6,2.689,3,4.02,NaN,"1,407.27","26,023.24","29,245.77","-3,222.53","-3,233.26","-2,116.07",NaN,"51,791.90","95,199.40",170.037,76.7,"1,439,323,776",153.0,38.0,61.0,NaN,1/26/2020,4.2,24.7,1.15,1.17,1.14,1.04,1.02,0.92,1.06,63.1,56.35,70.52
4984,4374.0,Anhui,China,31.8257,117.2264,2020-01-24,15.0,0.0,NaN,China/Anhui,24,6,0,8.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2,0,0,0,0,0,25.6,China,"89,473.25","95,199.40",106.4,"63,579.45","67,648.44",NaN,"27,272.31","19,379.63",19.191,3.491,42.674,43.9,125.931,123.888,NaN,3.877,3.6,2.689,3,4.02,NaN,"1,407.27","26,023.24","29,245.77","-3,222.53","-3,233.26","-2,116.07",NaN,"51,791.90","95,199.40",170.037,76.7,"1,439,323,776",153.0,38.0,61.0,NaN,1/26/2020,4.2,24.7,1.15,1.17,1.14,1.04,1.02,0.92,1.06,63.1,56.35,70.52
4985,4375.0,Anhui,China,31.8257,117.2264,2020-01-25,39.0,0.0,NaN,China/Anhui,25,24,0,6.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3,1,0,0,0,0,25.6,China,"89,473.25","95,199.40",106.4,"63,579.45","67,648.44",NaN,"27,272.31","19,379.63",19.191,3.491,42.674,43.9,125.931,123.888,NaN,3.877,3.6,2.689,3,4.02,NaN,"1,407.27","26,023.24","29,245.77","-3,222.53","-3,233.26","-2,116.07",NaN,"51,791.90","95,199.40",170.037,76.7,"1,439,323,776",153.0,38

In [271]:
df_traintest7[df_traintest7["place_id"]=="China/Hubei"].head()

,Id,Province/State,Country/Region,Lat,Long,Date,ConfirmedCases,Fatalities,ForecastId,place_id,day,cases/day,fatal/day,cases/day-(1-1),cases/day-(1-7),cases/day-(8-14),cases/day-(15-21),fatal/day-(1-1),fatal/day-(1-7),fatal/day-(8-14),fatal/day-(15-21),days_since_1cases,days_since_10cases,days_since_100cases,days_since_1fatal,days_since_10fatal,days_since_100fatal,SmokingRate,Country,Gross_domestic_product__constant_prices,Gross_domestic_product__current_prices,Gross_domestic_product__deflator,Gross_domestic_product_per_capita__constant_prices,Gross_domestic_product_per_capita__current_prices,Output_gap_in_percent_of_potential_GDP,Gross_domestic_product_based_on_purchasing_power_parity__PPP__valuation_of_country_GDP,Gross_domestic_product_based_on_purchasing_power_parity__PPP__per_capita_GDP,Gross_domestic_product_based_on_purchasing_power_parity__PPP__share_of_world_total,Implied_PPP_conversion_rate,Total_investment,Gross_national_savings,Inflation__average_consumer_prices,Inflation__end_of_period_consumer_prices,Six_month_London_interbank_offered_rate__LIBOR_,Volume_of_imports_of_goods_and_services,Volume_of_Imports_of_goods,Volume_of_exports_of_goods_and_services,Volume_of_exports_of_goods,Unemployment_rate,Employment,Population,General_government_revenue,General_government_total_expenditure,General_government_net_lending_borrowing,General_government_structural_balance,General_government_primary_net_lending_borrowing,General_government_net_debt,General_government_gross_debt,Gross_domestic_product_corresponding_to_fiscal_year__current_prices,Current_account_balance,LifeExpectancy,pop,density,medianage,urbanpop,quarantine,schools,hospibed,smokers,sex0,sex14,sex25,sex54,sex64,sex65plus,sexratio,lung,femalelung,malelung
6360,5581.0,Hubei,China,30.9756,112.2707,2020-01-22,444.0,17.0,NaN,China/Hubei,22,444,17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,35,22,4,13,0,0,25.6,China,"89,473.25","95,199.40",106.4,"63,579.45","67,648.44",NaN,"27,272.31","19,379.63",19.191,3.491,42.674,43.9,125.931,123.888,NaN,3.877,3.6,2.689,3,4.02,NaN,"1,407.27","26,023.24","29,245.77","-3,222.53","-3,233.26","-2,116.07",NaN,"51,791.90","95,199.40",170.037,76.7,"1,439,323,776",153.0,38.0,61.0,NaN,1/26/2020,4.2,24.7,1.15,1.17,1.14,1.04,1.02,0.92,1.06,63.1,56.35,70.52
6361,5582.0,Hubei,China,30.9756,112.2707,2020-01-23,444.0,17.0,NaN,China/Hubei,23,0,0,444.0,0.0,0.0,0.0,17.0,0.0,0.0,0.0,36,23,5,14,1,0,25.6,China,"89,473.25","95,199.40",106.4,"63,579.45","67,648.44",NaN,"27,272.31","19,379.63",19.191,3.491,42.674,43.9,125.931,123.888,NaN,3.877,3.6,2.689,3,4.02,NaN,"1,407.27","26,023.24","29,245.77","-3,222.53","-3,233.26","-2,116.07",NaN,"51,791.90","95,199.40",170.037,76.7,"1,439,323,776",153.0,38.0,61.0,NaN,1/26/2020,4.2,24.7,1.15,1.17,1.14,1.04,1.02,0.92,1.06,63.1,56.35,70.52
6362,5583.0,Hubei,China,30.9756,112.2707,2020-01-24,549.0,24.0,NaN,China/Hubei,24,105,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37,24,6,15,2,0,25.6,China,"89,473.25","95,199.40",106.4,"63,579.45","67,648.44",NaN,"27,272.31","19,379.63",19.191,3.491,42.674,43.9,125.931,123.888,NaN,3.877,3.6,2.689,3,4.02,NaN,"1,407.27","26,023.24","29,245.77","-3,222.53","-3,233.26","-2,116.07",NaN,"51,791.90","95,199.40",170.037,76.7,"1,439,323,776",153.0,38.0,61.0,NaN,1/26/2020,4.2,24.7,1.15,1.17,1.14,1.04,1.02,0.92,1.06,63.1,56.35,70.52
6363,5584.0,Hubei,China,30.9756,112.2707,2020-01-25,761.0,40.0,NaN,China/Hubei,25,212,16,105.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,38,25,7,16,3,0,25.6,China,"89,473.25","95,199.40",106.4,"63,579.45","67,648.44",NaN,"27,272.31","19,379.63",19.191,3.491,42.674,43.9,125.931,123.888,NaN,3.877,3.6,2.689,3,4.02,NaN,"1,407.27","26,023.24","29,245.77","-3,222.53","-3,233.26","-2,116.07",NaN,"51,791.90","95,199.40",170.037,76.7,"1,439,323,776",153.0,38.0,61.0,NaN,1/26/2020,4.2,24.7,1.15,1.17,1.14,1.04,1.02,0.92,1.06,63.1,56.35,70.52
6364,5585.0,Hubei,China,30.9756,112.2707,2020-01-26,1058.0,52.0,NaN,China/Hubei,26,297,12,212.0,0.0,0.0,0.0,16.0,0.0,0.0,0.0,39,26,8,17,4,0,25.6,China,"89,473.25","95,199.

In [218]:
#############################一段比较复杂的处理代码
#############################主要的处理逻辑：
#######给值存在的每个地区,国家赋予一个独一无二的地区标签

In [272]:
def encode_label(df, col, freq_limit=0):
    df[col][pd.isna(df[col])] = 'nan'
    tmp = df[col].value_counts()
    cols = tmp.index.values               ######################cols是索引值
    freq = tmp.values                   ######################freq是对应的值
    num_cols = (freq>=freq_limit).sum()
    print("col: {}, num_cat: {}, num_reduced: {}".format(col, len(cols), num_cols))
    col_new = '{}_le'.format(col)
    df_new = pd.DataFrame(np.ones(len(df), np.int16)*(num_cols-1), columns=[col_new])
    for i, item in enumerate(cols[:num_cols]):
        df_new[col_new][df[col]==item] = i
    return df_new

def get_df_le(df, col_index, col_cat):
    df_new = df[[col_index]]
    for col in col_cat:
        df_tmp = encode_label(df, col)
        df_new = pd.concat([df_new, df_tmp], axis=1)
    return df_new

df_traintest7['id'] = np.arange(len(df_traintest7))
df_le = get_df_le(df_traintest7, 'id', ['Country/Region', 'Province/State'])
df_traintest8 = pd.merge(df_traintest7, df_le, on='id', how='left')

F:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


col: Country/Region, num_cat: 163, num_reduced: 163
col: Province/State, num_cat: 129, num_reduced: 129


In [273]:
df_traintest8['cases/day'] = df_traintest8['cases/day'].astype(np.float)
df_traintest8['fatal/day'] = df_traintest8['fatal/day'].astype(np.float)

In [274]:
df_traintest8.head()

,Id,Province/State,Country/Region,Lat,Long,Date,ConfirmedCases,Fatalities,ForecastId,place_id,day,cases/day,fatal/day,cases/day-(1-1),cases/day-(1-7),cases/day-(8-14),cases/day-(15-21),fatal/day-(1-1),fatal/day-(1-7),fatal/day-(8-14),fatal/day-(15-21),days_since_1cases,days_since_10cases,days_since_100cases,days_since_1fatal,days_since_10fatal,days_since_100fatal,SmokingRate,Country,Gross_domestic_product__constant_prices,Gross_domestic_product__current_prices,Gross_domestic_product__deflator,Gross_domestic_product_per_capita__constant_prices,Gross_domestic_product_per_capita__current_prices,Output_gap_in_percent_of_potential_GDP,Gross_domestic_product_based_on_purchasing_power_parity__PPP__valuation_of_country_GDP,Gross_domestic_product_based_on_purchasing_power_parity__PPP__per_capita_GDP,Gross_domestic_product_based_on_purchasing_power_parity__PPP__share_of_world_total,Implied_PPP_conversion_rate,Total_investment,Gross_national_savings,Inflation__average_consumer_prices,Inflation__end_of_period_consumer_prices,Six_month_London_interbank_offered_rate__LIBOR_,Volume_of_imports_of_goods_and_services,Volume_of_Imports_of_goods,Volume_of_exports_of_goods_and_services,Volume_of_exports_of_goods,Unemployment_rate,Employment,Population,General_government_revenue,General_government_total_expenditure,General_government_net_lending_borrowing,General_government_structural_balance,General_government_primary_net_lending_borrowing,General_government_net_debt,General_government_gross_debt,Gross_domestic_product_corresponding_to_fiscal_year__current_prices,Current_account_balance,LifeExpectancy,pop,density,medianage,urbanpop,quarantine,schools,hospibed,smokers,sex0,sex14,sex25,sex54,sex64,sex65plus,sexratio,lung,femalelung,malelung,id,Country/Region_le,Province/State_le
0,1.0,nan,Afghanistan,33.0,65.0,2020-01-22,0.0,0.0,NaN,Afghanistan,22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,20.475517,Afghanistan,528.154,"1,692.57",320.469,"14,300.19","45,827.64",NaN,75.87,"2,054.23",0.053,22.309,18.346,20.174,124.809,131.29,NaN,3.559,2.773,13.252,17.026,NaN,NaN,36.933,469.79,467.62,2.17,NaN,2.532,NaN,101.669,"1,692.57",0.425,64.5,"38,928,346",60.0,18.0,25.0,NaN,NaN,0.5,NaN,1.05,1.03,1.03,1.03,0.97,0.85,1.03,37.62,36.31,39.33,0,69,0
1,2.0,nan,Afghanistan,33.0,65.0,2020-01-23,0.0,0.0,NaN,Afghanistan,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,20.475517,Afghanistan,528.154,"1,692.57",320.469,"14,300.19","45,827.64",NaN,75.87,"2,054.23",0.053,22.309,18.346,20.174,124.809,131.29,NaN,3.559,2.773,13.252,17.026,NaN,NaN,36.933,469.79,467.62,2.17,NaN,2.532,NaN,101.669,"1,692.57",0.425,64.5,"38,928,346",60.0,18.0,25.0,NaN,NaN,0.5,NaN,1.05,1.03,1.03,1.03,0.97,0.85,1.03,37.62,36.31,39.33,1,69,0
2,3.0,nan,Afghanistan,33.0,65.0,2020-01-24,0.0,0.0,NaN,Afghanistan,24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,20.475517,Afghanistan,528.154,"1,692.57",320.469,"14,300.19","45,827.64",NaN,75.87,"2,054.23",0.053,22.309,18.346,20.174,124.809,131.29,NaN,3.559,2.773,13.252,17.026,NaN,NaN,36.933,469.79,467.62,2.17,NaN,2.532,NaN,101.669,"1,692.57",0.425,64.5,"38,928,346",60.0,18.0,25.0,NaN,NaN,0.5,NaN,1.05,1.03,1.03,1.03,0.97,0.85,1.03,37.62,36.31,39.33,2,69,0
3,4.0,nan,Afghanistan,33.0,65.0,2020-01-25,0.0,0.0,NaN,Afghanistan,25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,20.475517,Afghanistan,528.154,"1,692.57",320.469,"14,300.19","45,827.64",NaN,75.87,"2,054.23",0.053,22.309,18.346,20.174,124.809,131.29,NaN,3.559,2.773,13.252,17.026,NaN,NaN,36.933,469.79,467.62,2.17,NaN,2.532,NaN,101.669,"1,692.57",0.425,64.5,"38,928,346",60.0,18.0,25.0,NaN,NaN,0.5,NaN,1.05,1.03,1.03,1.03,0.97,0.85,1.03,37.62,36.31,39.33,3,69,0
4,5.0,nan,Afghanistan,33.0,65.0,2020-01-26,0.0,0.0,NaN,Afghanistan,26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,20.475517,Afghanistan,528.154,"1,692.57",320.469,"14,300.19","45,827.64",NaN,75.87,"2,054.23",0.053,22.309,18.346,20.174,124.809,131.29,NaN,3.559,2.773,13.252,17.026,NaN,NaN,36.933,469.79,467.62,2.17,NaN,2.532,N

In [275]:
# covert object type to float
def func(x):
    x_new = 0
    try:
        x_new = float(x.replace(",", ""))
    except:
        x_new = np.nan
    return x_new
cols = [
    'Gross_domestic_product__constant_prices', 
    'Gross_domestic_product__current_prices', 
    'Gross_domestic_product__deflator', 
    'Gross_domestic_product_per_capita__constant_prices', 
    'Gross_domestic_product_per_capita__current_prices', 
    'Output_gap_in_percent_of_potential_GDP', 
    'Gross_domestic_product_based_on_purchasing_power_parity__PPP__valuation_of_country_GDP', 
    'Gross_domestic_product_based_on_purchasing_power_parity__PPP__per_capita_GDP', 
    'Gross_domestic_product_based_on_purchasing_power_parity__PPP__share_of_world_total', 
    'Implied_PPP_conversion_rate', 'Total_investment', 
    'Gross_national_savings', 'Inflation__average_consumer_prices', 
    'Inflation__end_of_period_consumer_prices', 
    'Six_month_London_interbank_offered_rate__LIBOR_', 
    'Volume_of_imports_of_goods_and_services', 
    'Volume_of_Imports_of_goods', 
    'Volume_of_exports_of_goods_and_services', 
    'Volume_of_exports_of_goods', 'Unemployment_rate', 'Employment', 'Population', 
    'General_government_revenue', 'General_government_total_expenditure', 
    'General_government_net_lending_borrowing', 'General_government_structural_balance', 
    'General_government_primary_net_lending_borrowing', 'General_government_net_debt', 
    'General_government_gross_debt', 'Gross_domestic_product_corresponding_to_fiscal_year__current_prices', 
    'Current_account_balance', 'pop'
]
for col in cols:
    df_traintest8[col] = df_traintest8[col].apply(lambda x: func(x))  
print(df_traintest8['pop'].dtype)

float64


In [276]:
df_traintest8[df_traintest8['place_id']=='China/Hubei'].head()

,Id,Province/State,Country/Region,Lat,Long,Date,ConfirmedCases,Fatalities,ForecastId,place_id,day,cases/day,fatal/day,cases/day-(1-1),cases/day-(1-7),cases/day-(8-14),cases/day-(15-21),fatal/day-(1-1),fatal/day-(1-7),fatal/day-(8-14),fatal/day-(15-21),days_since_1cases,days_since_10cases,days_since_100cases,days_since_1fatal,days_since_10fatal,days_since_100fatal,SmokingRate,Country,Gross_domestic_product__constant_prices,Gross_domestic_product__current_prices,Gross_domestic_product__deflator,Gross_domestic_product_per_capita__constant_prices,Gross_domestic_product_per_capita__current_prices,Output_gap_in_percent_of_potential_GDP,Gross_domestic_product_based_on_purchasing_power_parity__PPP__valuation_of_country_GDP,Gross_domestic_product_based_on_purchasing_power_parity__PPP__per_capita_GDP,Gross_domestic_product_based_on_purchasing_power_parity__PPP__share_of_world_total,Implied_PPP_conversion_rate,Total_investment,Gross_national_savings,Inflation__average_consumer_prices,Inflation__end_of_period_consumer_prices,Six_month_London_interbank_offered_rate__LIBOR_,Volume_of_imports_of_goods_and_services,Volume_of_Imports_of_goods,Volume_of_exports_of_goods_and_services,Volume_of_exports_of_goods,Unemployment_rate,Employment,Population,General_government_revenue,General_government_total_expenditure,General_government_net_lending_borrowing,General_government_structural_balance,General_government_primary_net_lending_borrowing,General_government_net_debt,General_government_gross_debt,Gross_domestic_product_corresponding_to_fiscal_year__current_prices,Current_account_balance,LifeExpectancy,pop,density,medianage,urbanpop,quarantine,schools,hospibed,smokers,sex0,sex14,sex25,sex54,sex64,sex65plus,sexratio,lung,femalelung,malelung,id,Country/Region_le,Province/State_le
6360,5581.0,Hubei,China,30.9756,112.2707,2020-01-22,444.0,17.0,NaN,China/Hubei,22,444.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,35,22,4,13,0,0,25.6,China,89473.25,95199.4,106.4,63579.45,67648.44,NaN,27272.31,19379.63,19.191,3.491,42.674,43.9,125.931,123.888,NaN,3.877,3.6,2.689,3.0,4.02,NaN,1407.27,26023.24,29245.77,-3222.53,-3233.26,-2116.07,NaN,51791.9,95199.4,170.037,76.7,1.439324e+09,153.0,38.0,61.0,NaN,1/26/2020,4.2,24.7,1.15,1.17,1.14,1.04,1.02,0.92,1.06,63.1,56.35,70.52,6360,1,123
6361,5582.0,Hubei,China,30.9756,112.2707,2020-01-23,444.0,17.0,NaN,China/Hubei,23,0.0,0.0,444.0,0.0,0.0,0.0,17.0,0.0,0.0,0.0,36,23,5,14,1,0,25.6,China,89473.25,95199.4,106.4,63579.45,67648.44,NaN,27272.31,19379.63,19.191,3.491,42.674,43.9,125.931,123.888,NaN,3.877,3.6,2.689,3.0,4.02,NaN,1407.27,26023.24,29245.77,-3222.53,-3233.26,-2116.07,NaN,51791.9,95199.4,170.037,76.7,1.439324e+09,153.0,38.0,61.0,NaN,1/26/2020,4.2,24.7,1.15,1.17,1.14,1.04,1.02,0.92,1.06,63.1,56.35,70.52,6361,1,123
6362,5583.0,Hubei,China,30.9756,112.2707,2020-01-24,549.0,24.0,NaN,China/Hubei,24,105.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37,24,6,15,2,0,25.6,China,89473.25,95199.4,106.4,63579.45,67648.44,NaN,27272.31,19379.63,19.191,3.491,42.674,43.9,125.931,123.888,NaN,3.877,3.6,2.689,3.0,4.02,NaN,1407.27,26023.24,29245.77,-3222.53,-3233.26,-2116.07,NaN,51791.9,95199.4,170.037,76.7,1.439324e+09,153.0,38.0,61.0,NaN,1/26/2020,4.2,24.7,1.15,1.17,1.14,1.04,1.02,0.92,1.06,63.1,56.35,70.52,6362,1,123
6363,5584.0,Hubei,China,30.9756,112.2707,2020-01-25,761.0,40.0,NaN,China/Hubei,25,212.0,16.0,105.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,38,25,7,16,3,0,25.6,China,89473.25,95199.4,106.4,63579.45,67648.44,NaN,27272.31,19379.63,19.191,3.491,42.674,43.9,125.931,123.888,NaN,3.877,3.6,2.689,3.0,4.02,NaN,1407.27,26023.24,29245.77,-3222.53,-3233.26,-2116.07,NaN,51791.9,95199.4,170.037,76.7,1.439324e+09,153.0,38.0,61.0,NaN,1/26/2020,4.2,24.7,1.15,1.17,1.14,1.04,1.02,0.92,1.06,63.1,56.35,70.52,6363,1,123
6364,5585.0,Hubei,China,30.9756,112.2707,2020-01-26,1058.0,52.0,NaN,China/Hubei,26,297.0,12.0,212.0,0.0,0.0,0.0,16.0,0.0,0.0,0.0,39,26,8,17,4,0,25.6,China,89473.25,95199.4,106.4,63579.45,67648.44,NaN,27272.31,19379.63,19.191,3.491,42.674,43.9,125.931,123.8

开始训练模型